In [4]:
@!python -m spacy download pt_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.7.0/pt_core_news_sm-3.7.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [5]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

import spacy

gpu = spacy.prefer_gpu()
print(gpu)

True


In [6]:
# download e unzip ==> http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip
w2v_modelo_cbow = KeyedVectors.load_word2vec_format(
    "../introducao-word-embedding/cbow_s300.txt"
)

In [7]:
# download e unzip ==> http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s300.zip
w2v_modelo_sg = KeyedVectors.load_word2vec_format(
    "../introducao-word-embedding/skip_s300.txt"
)

In [8]:
# download ==> https://cdn3.gnarususercontent.com.br/1638-word-embedding/treino.csv
artigo_treino = pd.read_csv("../introducao-word-embedding/treino.csv")

In [9]:
# download ==> https://cdn3.gnarususercontent.com.br/1638-word-embedding/teste.csv
artigo_teste = pd.read_csv("../introducao-word-embedding/teste.csv")

In [11]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])


def tokenizador(texto):

    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return tokens_validos


texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [12]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1, 300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass

    return vetor_resultante


vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-0.11220199 -0.507946   -0.99369503  0.067104    0.871621   -0.28742
  -0.03983801  0.54327301  0.24698202  0.83912901 -1.069877    0.55167699
   0.187593    0.090007    0.098912    0.38884801 -0.49045099 -0.03031399
  -0.218182    1.42319699 -0.39475699 -0.04351101 -0.063719   -0.212974
   0.200938   -0.215609    0.08405001 -0.44516101 -0.00246099  0.37024
   0.445444   -1.10328901  0.24673     0.14532401 -0.23344    -0.408868
  -0.284244    0.39413801  0.57867099  0.417927   -0.062123    0.082006
  -0.517467    0.08302799 -0.460177    0.35803901  0.16759    -0.732339
  -0.507158    0.02639301  0.14691799 -0.48757798  0.70211802  0.18344299
  -0.121941    0.276907   -0.388653   -0.511082   -0.63674802 -0.02843899
   0.27211098 -0.31658901 -0.17327601  0.086747    0.10449401  0.70350702
  -0.19605199 -0.22203901 -0.471284    0.19785901  0.033516   -0.65221997
   0.077695   -0.470488    0.51082799  0.01838199  0.041174    1.24306798
  -0.75908899  0.350811    0.45493601  0.88

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz


matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter=800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)

    return RL


RL_cbow = classificador(
    w2v_modelo_cbow,
    matriz_vetores_treino_cbow,
    artigo_treino.category,
    matriz_vetores_teste_cbow,
    artigo_teste.category,
)

In [ ]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(
    w2v_modelo_sg,
    matriz_vetores_treino_sg,
    artigo_treino.category,
    matriz_vetores_teste_sg,
    artigo_teste.category,
)

In [ ]:
import pickle

with open("/content/drive/My Drive/curso_word2vec/rl_cbow.pkl", "wb") as f:
  pickle.dump(RL_cbow, f)

In [ ]:
with open("/content/drive/My Drive/curso_word2vec/rl_sg.pkl", "wb") as f:
  pickle.dump(RL_sg, f)